In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
 #   for filename in filenames:
  #      print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
train_dir = '/kaggle/input/fer2013plus/fer2013plus/fer2013/train'
test_dir = '/kaggle/input/fer2013plus/fer2013plus/fer2013/test'


In [6]:
from keras.src.legacy.preprocessing.image import ImageDataGenerator

In [15]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)
test_datagen = ImageDataGenerator(rescale=1./255)

In [17]:
train_generator= train_datagen.flow_from_directory('/kaggle/input/fer2013plus/fer2013plus/fer2013/train',
    target_size=(48,48),
    color_mode='grayscale',
    class_mode='categorical',
    batch_size=100,
    shuffle=True,
   )
test_generator= test_datagen.flow_from_directory('/kaggle/input/fer2013plus/fer2013plus/fer2013/test',
    target_size=(48,48),
    color_mode='grayscale',
    class_mode='categorical',
    batch_size=100,
    shuffle=True,
   )

Found 28386 images belonging to 8 classes.
Found 7099 images belonging to 8 classes.


In [18]:
from keras.models import Sequential, load_model
from keras.layers import Conv2D,GlobalAveragePooling2D,MaxPooling2D,Flatten,Dropout,BatchNormalization,Dense

In [19]:
input_shape = (48, 48, 1)

In [21]:
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(BatchNormalization())

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())

model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))


model.add(Conv2D(64, (3, 3), padding="same", activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(96, (3, 3), dilation_rate=(2, 2), activation='relu', padding="same"))
model.add(Conv2D(96, (3, 3), padding="same", activation='relu'))



model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(8, activation='softmax'))

In [22]:
model.compile(loss="categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), metrics=['accuracy'])

In [23]:
history = model.fit(
    train_generator,
    steps_per_epoch=100,
    epochs=50,
    validation_data=val_generator,
    validation_steps=len(val_generator)
)

Epoch 1/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 46s 310ms/step - accuracy: 0.1617 - loss: 2.2424 - val_accuracy: 0.3625 - val_loss: 2.0205
Epoch 2/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 20s 201ms/step - accuracy: 0.2550 - loss: 2.0179 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 3/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 40s 403ms/step - accuracy: 0.3448 - loss: 1.8941 - val_accuracy: 0.3689 - val_loss: 1.8897
Epoch 4/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 22s 205ms/step - accuracy: 0.3702 - loss: 1.8152 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 5/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 30s 299ms/step - accuracy: 0.4105 - loss: 1.7288 - val_accuracy: 0.4293 - val_loss: 1.7129
Epoch 6/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 16s 157ms/step - accuracy: 0.4477 - loss: 1.6288 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 7/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 31s 294ms/step - accuracy: 0.4720 - loss: 1.5555 - val_accuracy: 0.5139 - val_loss: 1.4478
Epoch 8/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 20s 200ms/step - ac

In [24]:
loss,accuracy = model.evaluate(test_generator, verbose=1)
print('accuracy',accuracy)

71/71 ━━━━━━━━━━━━━━━━━━━━ 71s 1s/step - accuracy: 0.7427 - loss: 0.7441 
accuracy 0.7403859496116638


In [26]:
from sklearn.metrics import f1_score
y_pred = model.predict(test_generator).argmax(axis=1)
y_true = test_generator.classes
f1 = f1_score(y_true,y_pred,average='weighted')
print('f1 score',f1)

71/71 ━━━━━━━━━━━━━━━━━━━━ 13s 168ms/step
f1 score 0.2510836587659747
